In [1]:
#Import modules for working with real numbers
import pandas as pd
import numpy as np
import math
from scipy import stats
import random

#Import modules for working with plots
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits.mplot3d.axes3d as p3

#Useful to view the plots neatly together
%matplotlib inline

In [2]:
"""
This is the k-armed bandit algorithm as outlined in 
pseudocode from Reinforcement Learning: An Introduction - Sutton, Barto.

Basically, it's useful to think of the algorithm as if one
were playing a 10-levered, or k-levered, slot machine without knowing 
in advance the reward received from any of the levers, with the goal of 
maximizing the reward received. As we are interacting with
the environment of the simple game, how do we choose which lever and
keep track of the rewards we've recieved from prior pulls?

This algorithm balances exploration and exploitation in such a game,
as the balance between exploring a new option that could potentially
have a higher reward and exploiting an option that is estimated to
return the largest reward. With a stationary reward distribution, 
the value estimates for the actions will converge with a sufficiently
large number of time steps, t, to their actual values 
with a value estimate update as the sample mean reward received.

However, if the rewards' underlying distribution is considered to 
be nonstationary, as potentially changing over time, then we can 
alter the learning rate to give proportionally more weight in our 
value estimates to recent rewards. If this is the case, then we can 
use an exponential-recency weighted learning rate 
for updating the value estimates.
"""

#initialize the number of actions available to be selected, eg. k-armed bandit
k = range(10)
#initialize the number of action selections, or time steps, in a run
t = range(1000)
#initialize the number of runs as the size of the bandit problem testbed
runs = range(30)

#initialize the level of epsilon-greedy action selection
epsilon = 0.1

#initialize this value as True if the rewards' underlying
#distribution is considered to be changing over time,
#as nonstationary, which will alter the learning rate
nonstationary = False
#initialize the learning rate
alpha = 0.1

###

#bandit testbed simulation loop
for run in runs:
    
#initialize the initial estimated values as expected rewards
    q_est = [0 for i in k]
#initialize action selection counter for reward sample mean
    n_act = [0 for i in k]
    
#initialize the actual values
    q_val = [np.random.normal() for i in k]
    
#initialize sets of random variables to be used in action selection
    rn1 = [random.random() for r in t]
    rn2 = [random.random() for r in t]
    rn3 = [random.random() for r in t]
    
#loop through a run's timesteps for t action selections
    for x in t:

#Choose an action from the action set.
#With probability 1-epsilon, choose the greedy action,
#by taking note of all actions that are estimated to 
#have the maximum value, and in the case that there are ties,
#choose between the tied maximum estimates randomly.
        if rn1[x] < (1-epsilon):
            max_ind = [ind for ind, q in enumerate(q_est) if q == max(q_est)]
            max_i_r = int(math.floor(rn2[x]*len(max_ind)))
            j = max_ind[max_i_r]
#With probability epsilon, choose a random action.
        if rn1[x] >= (1-epsilon):
            j = int(math.floor(rn3[x]*len(q_est)))
        
#The actual reward of the chosen action is now
#actually assigned as a normally distributed random variable
#with mean equivalent to the actual value of the chosen action,
#which was earlier assigned a standard normal value, and the
#standard deviation the same as in standard normal.
        r_t = np.random.normal(loc=q_val[j])
    
#increment the action selection counter for the chosen action
        n_act[j] += 1
#increment the selected action's estimated value based on the reward
#as an exponential-recency weighted average, learning at rate alpha
#if we suspect the distribution of the reward is nonstationary
        if nonstationary is True:
            q_est[j] += alpha*(r_t - q_est[j])
#incrementally update the action's estimated value as the 
#sample mean reward received if the rewards' distributions are stationary
        else:
            q_est[j] += (1/n_act[j])*(r_t - q_est[j])
        
    

    print(n_act)
    q_est = ([round(q, 5) for q in q_est])
    print(q_est)
    q_val = ([round(q, 5) for q in q_val])
    print(q_val)

[9, 10, 10, 11, 11, 11, 908, 5, 9, 16]
[1.09082, -0.92372, -0.26708, 0.93949, -1.70982, 0.75095, 2.06479, -1.16624, 0.04234, 1.11335]
[1.01867, -0.81497, -0.29979, 1.58053, -1.69311, 0.58928, 2.09238, -0.77127, -0.46282, 1.08585]
[17, 12, 6, 13, 12, 9, 11, 227, 685, 8]
[-0.12077, -1.34454, -1.39447, -0.29447, -0.58302, -0.63977, -1.33399, -0.04544, 0.61302, -0.17557]
[0.08652, -1.12167, -1.32223, -0.0505, -0.29272, -0.05389, -1.14197, 0.08576, 0.61229, -0.0778]
[8, 546, 7, 9, 371, 10, 8, 23, 16, 2]
[-0.49909, 0.61291, -0.97647, -1.74025, 1.03067, -0.27908, -0.48143, 0.47695, -0.66319, 0.81773]
[-1.10212, 0.57791, -1.1163, -0.90978, 1.0847, -0.45994, -0.06492, 0.54666, -0.78152, 0.25006]
[18, 12, 11, 8, 8, 905, 11, 10, 8, 9]
[0.60394, -1.22958, -0.31889, 0.05616, 1.07437, 1.08101, -1.24872, -0.33332, -1.74924, -0.26893]
[0.32854, -0.81928, -0.28658, 0.55396, 1.13482, 1.08651, -1.12761, -1.13229, -1.87798, -0.4474]
[41, 11, 13, 10, 862, 26, 6, 8, 11, 12]
[0.30478, 0.39192, -1.50592, -0.6